In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from ttkthemes import ThemedStyle
import pandas as pd
import geopy
from tkinter import ttk
import json
import time
import selenium
from selenium import webdriver
import random
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import geopy.distance
from geopy.geocoders import Nominatim

In [2]:
def getCoordinates(address):
    try:
        geolocator = Nominatim(user_agent="Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)")
        loc = geolocator.geocode(address)
        coordinates = (loc.latitude,loc.longitude)
    except:
        coordinates = (0,0)
    return coordinates


In [5]:
def load_data():
    # Open a file dialog to select the JSON file
    print(address_entry.get())
    address = address_entry.get()
    coordinates = getCoordinates(address)
    df = pd.read_json("./allRestos.json")

        # Read JSON data into a DataFrame

    df["Tags"] = df["Tags"].astype(str)
    df["Tags"] = df.apply(lambda row: row.Tags.replace("[",""), axis=1)
    df["Tags"] = df.apply(lambda row: row.Tags.replace("]",""), axis=1)
    df["Tags"] = df.apply(lambda row: row.Tags.replace("'"," "), axis=1)
    df["Tags"] = df.apply(lambda row: row.Tags.replace(","," "), axis=1)
    df.Rating = pd.to_numeric(df.Rating, errors='coerce')
    dataset = df
    selected_cuisine = cuisine_var.get()
    selected_rating = rating_var.get()

        # Filter the DataFrame based on selected values
        #filtered_df = globalDF[(globalDF['Tags'] == selected_cuisine) & (globalDF['rating'] == selected_rating)]
    filtered_df = dataset[(dataset["Tags"].str.contains(selected_cuisine)) & (dataset["Rating"]>int(selected_rating))]
    
      # Remove duplicates based on the restaurant's name
    filtered_df = filtered_df.drop_duplicates(subset='Name')
    
    filtered_df["Rating"]  = filtered_df.apply(lambda row: str(row.Rating)+ " /10",axis=1)
    filtered_df["Distance"]  = filtered_df.apply(lambda row: str(round(geopy.distance.geodesic(row.Coordinates,coordinates).kilometers,2)) + "km",axis=1)
    filtered_df= filtered_df.sort_values(by = "Distance")[:10]
        # # Display the filtered results in a text widget
        # result_text.delete(1.0, tk.END)  # Clear previous results
        # result_text.insert(tk.END, filtered_df.to_string(index=False))
    for row in result_tree.get_children():
        result_tree.delete(row)

        # Display the filtered results in the treeview as a table
    for index, row in filtered_df.iterrows():
        result_tree.insert("", "end", values=(row['Name'], row['Tags'], row['Rating'], row['Distance']))


# Create the main Tkinter window
root = tk.Tk()
root.title("LuxResto")  # Set window title to "LuxResto"

# Set background color and font style
root.configure(bg="#3CA0D0")

font_style = "Georgia"

# Create a Frame for the controls
control_frame = tk.Frame(root, bg="#FFFFFF", padx=20, pady=20, bd=2, relief=tk.RIDGE)
control_frame.pack(padx=10, pady=10)

# Title
title_label = tk.Label(control_frame, text="LuxResto", font=(font_style, 24, "underline"), bg="#FFFFFF")
title_label.grid(row=0, column=0, columnspan=2, pady=10)

# Load JSON button 
load_button = tk.Button(control_frame, text="Search", command=load_data, bg="#FF0000", fg="white", font=(font_style, 16), bd=2, relief=tk.RAISED)
load_button.grid(row=4, column=1, padx=5, pady=10, sticky="ew")

# Cuisine label and dropdown 
cuisine_label = tk.Label(control_frame, text="Select Cuisine:", font=(font_style, 14), bg="#FFFFFF")
cuisine_label.grid(row=1, column=0, padx=5, pady=5)
cuisine_var = tk.StringVar()
cuisine_options = ["Indienne", "Népalaise", "Luxembourgeoise", "Chinoise", "Asiatique", "Française", "Italienne", "Japanese", "Buffet", "Burgers", "Mexicaine", "Pizzeria", "Végétarienne","Vegan", "Libanaise","Portugaise","Thaïlandaise"]
cuisine_options.sort()  # Sort the options alphabetically
cuisine_dropdown = tk.OptionMenu(control_frame, cuisine_var, *cuisine_options)  # Use * to unpack the list
cuisine_dropdown.config(font=(font_style, 12))
cuisine_dropdown.grid(row=1, column=1, padx=5, pady=5)
cuisine_var.set("Indienne")  # Set a default value

# Rating label and scale 
rating_label = tk.Label(control_frame, text="Select Rating:", font=(font_style, 14), bg="#FFFFFF")
rating_label.grid(row=2, column=0, padx=5, pady=5, sticky="w")
rating_var = tk.IntVar()
rating_scale = tk.Scale(control_frame, variable=rating_var, from_=0, to=10, orient=tk.HORIZONTAL, bg="black", fg="white", bd=2, relief=tk.RIDGE)
rating_scale.grid(row=2, column=1, padx=5, pady=5, sticky="ew")
rating_var.set(5)  # Set a default value

# Address label and entry field 
address_label = tk.Label(control_frame, text="Enter Address:", font=(font_style, 14), bg="#FFFFFF")
address_label.grid(row=3, column=0, padx=5, pady=5, sticky="w")

# Placeholder text
placeholder_text = "ex. 33, rue Camille Wampach"
address_entry = tk.Entry(control_frame, font=(font_style, 10, 'italic'), bd=2, relief=tk.RIDGE)
address_entry.insert(0, placeholder_text)  # Set the placeholder text
address_entry.grid(row=3, column=1, padx=5, pady=5, sticky="ew")

# Create a treeview to display the results as a table
result_tree = ttk.Treeview(root, columns=("Name", "Tags", "Rating", "Distance"), show="headings")
result_tree.heading("Name", text="Name")
result_tree.heading("Tags", text="Cuisine")
result_tree.heading("Rating", text="Rating")
result_tree.heading("Distance", text="Distance")
result_tree.pack(padx=10, pady=10)


# Start the Tkinter main loop
root.mainloop()
